<div class="align-center">
<a href="https://oumi.ai/"><img src="https://oumi.ai/docs/en/latest/_static/logo/header_logo.png" height="200"></a>

[![Documentation](https://img.shields.io/badge/Documentation-latest-blue.svg)](https://oumi.ai/docs/en/latest/index.html)
[![Discord](https://img.shields.io/discord/1286348126797430814?label=Discord)](https://discord.gg/oumi)
[![GitHub Repo stars](https://img.shields.io/github/stars/oumi-ai/oumi)](https://github.com/oumi-ai/oumi)
<a target="_blank" href="https://colab.research.google.com/github/oumi-ai/oumi/blob/main/notebooks/Oumi - Quantization Tutorial.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
</div>

👋 Welcome to Open Universal Machine Intelligence (Oumi)!

🚀 Oumi is a fully open-source platform that streamlines the entire lifecycle of foundation models - from [data preparation](https://oumi.ai/docs/en/latest/resources/datasets/datasets.html) and [training](https://oumi.ai/docs/en/latest/user_guides/train/train.html) to [evaluation](https://oumi.ai/docs/en/latest/user_guides/evaluate/evaluate.html) and [deployment](https://oumi.ai/docs/en/latest/user_guides/launch/launch.html). Whether you're developing on a laptop, launching large scale experiments on a cluster, or deploying models in production, Oumi provides the tools and workflows you need.

🤝 Make sure to join our [Discord community](https://discord.gg/oumi) to get help, share your experiences, and contribute to the project! If you are interested in joining one of the community's open-science efforts, check out our [open collaboration](https://oumi.ai/community) page.

⭐ If you like Oumi and you would like to support it, please give it a star on [GitHub](https://github.com/oumi-ai/oumi).

# Model Quantization Tutorial

This tutorial demonstrates how to use AWQ (Activation-aware Weight Quantization) to compress large language models while maintaining performance.

## Prerequisites

❗**NOTICE:** Model quantization requires a GPU. If running on Google Colab, you must use a GPU runtime (Colab Menu: `Runtime` -> `Change runtime type` -> Select `T4 GPU` or better).

⚠️ **DEVELOPMENT STATUS**: The quantization feature is currently under active development. Some features may change in future releases.

First, let's install Oumi with GPU support and the required quantization libraries:

```bash
pip install oumi[gpu]
pip install autoawq
pip install triton==3.0.0  # Required for AWQ inference compatibility
```

## 1. Basic AWQ Quantization

Let's start by quantizing TinyLlama to 4-bit using AWQ:

In [ ]:
from oumi.core.configs import QuantizationConfig, ModelParams  # type: ignore
from oumi.quantize import quantize  # type: ignore

# Configure quantization
config = QuantizationConfig(
    model=ModelParams(model_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0"),
    method="awq_q4_0",  # 4-bit AWQ quantization
    output_path="tinyllama_awq_4bit.safetensors",
    output_format="safetensors",  # Use SafeTensors format
    calibration_samples=512,  # Number of calibration samples
)

# Run quantization
print("Starting AWQ quantization...")
result = quantize(config)

# Calculate sizes and compression
original_size_gb = 2.2  # TinyLlama 1.1B in fp16
quantized_size_gb = result["quantized_size_bytes"] / (1024**3)  # type: ignore
compression_ratio = original_size_gb / quantized_size_gb

print("\n✅ Quantization complete!")
print(f"Original size (fp16): {original_size_gb:.2f}GB")
print(f"Quantized size (4-bit): {quantized_size_gb:.2f}GB")
print(f"Compression ratio: {compression_ratio:.1f}x")
size_reduction_pct = (original_size_gb - quantized_size_gb) / original_size_gb * 100
print(f"Size reduction: {size_reduction_pct:.1f}%")

## 2. Using the Quantized Model

Now let's load and use the quantized model for inference:

In [ ]:
from awq import AutoAWQForCausalLM  # type: ignore
from oumi.builders import build_tokenizer  # type: ignore
from oumi.core.configs import ModelParams  # type: ignore
import torch  # type: ignore

# Load the quantized model
model_path = "tinyllama_awq_4bit.safetensors"

print(f"Loading AWQ model from: {model_path}")
model = AutoAWQForCausalLM.from_quantized(
    model_path,
    fuse_layers=False,  # Disable layer fusion to avoid compatibility issues
    device_map="auto",
)

# Load tokenizer using oumi builder
tokenizer = build_tokenizer(
    ModelParams(model_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0")
)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print(f"✅ Model loaded! GPU memory: {torch.cuda.memory_allocated() / 1024**3:.2f}GB")

In [10]:
# Test inference
prompt = "Explain the benefits of model quantization in simple terms:"

# Tokenize
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

# Generate
print(f"Prompt: {prompt}\n")
print("Generating response...")

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=150,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.pad_token_id,
    )

# Decode and print response
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"Response:\n{response}")

Prompt: Explain the benefits of model quantization in simple terms:

Generating response...
Response:
Explain the benefits of model quantization in simple terms:

Model quantization is a technique that can achieve significant speed-up by reducing the amount of storage required to hold model parameters. In the context of deep learning, it reduces the storage needed to represent the weights of a model, which in turn reduces the size of the model and makes it easier to transport, store, and transfer. This helps with data reduction, which is crucial for real-time deployment scenarios. Moreover, model quantization can also lead to better utilization of processing capabilities of the GPU, since it reduces the memory requirements for storing model parameters. This reduces the amount of memory required to store and manipulate the model, and enables the usage of the entire memory capacity of the GPU. Overall, model quantization is an


## 3. Advanced Configuration

AWQ offers several configuration options for fine-tuning the quantization process:

In [ ]:
# Advanced AWQ configuration with more calibration samples
advanced_config = QuantizationConfig(
    model=ModelParams(model_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0"),
    method="awq_q4_0",
    output_path="tinyllama_awq_advanced.safetensors",
    output_format="safetensors",  # Use SafeTensors format
    # AWQ-specific parameters
    calibration_samples=1024,  # More samples for better calibration
    awq_group_size=128,  # Weight grouping size
    awq_version="GEMM",  # AWQ kernel version (GEMM is faster)
    awq_zero_point=True,  # Use zero-point quantization
)

print("Configuration:")
print(f"- Output format: {advanced_config.output_format}")
print(f"- Calibration samples: {advanced_config.calibration_samples}")
print(f"- Group size: {advanced_config.awq_group_size}")
print(f"- AWQ version: {advanced_config.awq_version}")
print(f"- Zero point: {advanced_config.awq_zero_point}")

## Summary

In this tutorial, you learned how to:

1. ✅ Quantize models using AWQ to 4-bit precision
2. ✅ Load and use AWQ quantized models for inference
3. ✅ Configure AWQ parameters for better quality


### Key Benefits of AWQ:
- **Memory Efficiency**: ~75% reduction in model size
- **Speed**: Faster inference due to reduced memory bandwidth
- **Quality**: Minimal performance degradation
- **Compatibility**: Works with most transformer models

Happy quantizing! 🚀